In [ ]:
using FixedSizeArrays
using LightGraphs
using Quaternions

In [ ]:
importall Base

In [ ]:
immutable CartesianFrame3D
    name::ASCIIString
end

immutable Point3D{T}
    frame::CartesianFrame3D
    v::Vec{3, T}
end

In [ ]:
function rotate{N, T}(x::Mat{3, N, T}, q::Quaternion{T})
    # TODO: efficiency?
    return Mat(rotationmatrix(q)) * x
end

immutable Transform3D{T}
    from::CartesianFrame3D
    to::CartesianFrame3D
    rot::Quaternion{T}
    trans::Vec{3, T}
end

Transform3D(from::CartesianFrame3D, to::CartesianFrame3D, rot::Quaternion) = Transform3D(from, to, rot, zero(Vec{3, real(typeof(rot))}))
Transform3D(from::CartesianFrame3D, to::CartesianFrame3D, trans::Vec{3}) = Transform3D(from, to, one(Quaternion{eltype(trans)}), trans)

function *(t1::Transform3D, t2::Transform3D)
    @assert t1.from == t2.to
    return Transform3D(t2.from, t1.to, t1.rot * t2.rot, t1.trans + rotate(t2.trans, t1.rot))
end

function *(t::Transform3D, point::Point3D)
    @assert t1.from == point.frame
    return Point3D(t.to, rotate(point.v, t.rot) + t.trans)
end

function inv{T}(t::Transform3D{T})
    rotinv = inv(t.rot)
    transinv = -rotate(t.trans, rotinv)
    return Transform3D(t.to, t.from, rotinv, transinv)
end

In [ ]:
f1 = CartesianFrame3D("1")
f2 = CartesianFrame3D("2")
f3 = CartesianFrame3D("3")
f4 = CartesianFrame3D("4")

In [ ]:
t1 = Transform3D(f2, f1, nquatrand(), rand(Vec{3, Float64}))
v = Point3D(f2, rand(Vec{3, Float64}))
w = t1 * v

In [ ]:
t2 = Transform3D(f3, f2, nquatrand())
t3 = t1 * t2

In [ ]:
t4 = Transform3D(f4, f3, Vec(1.0, 2.0, 3.0))
t5 = t3 * t4

In [ ]:
inv(t5) * t5

In [1]:
type TreeVertex{V, E}
    vertexData::V
    parent::Nullable{TreeVertex{V, E}}
    edgeToParentData::Nullable{E}
    children::Vector{TreeVertex{V, E}}
    
    TreeVertex{V}(vertexData::V) = new(vertexData, Nullable{TreeVertex{V, E}}(), Nullable{E}(), [])
    TreeVertex{V, E}(vertexData::V, parent::TreeVertex{V, E}, edgeData::E) = new(vertexData, parent, edgeData, [])
end
typealias Tree{V, E} TreeVertex{V, E}

TreeVertex{V,E}

In [2]:
isroot{V, E}(v::TreeVertex{V, E}) = isnull(v.parent)
isleaf{V, E}(v::TreeVertex{V, E}) = isempty(v.children)

function findfirst{V, E}(pred, tree::Tree{V, E})
    # depth first search
    root = tree
    pred(root) && return root
    
    for child in root.children
        vertex = findfirst(pred, child)
        vertex != nothing && return vertex
    end
    return nothing
end

function find{V, E}(pred, tree::Tree{V, E}, result = Vector{TreeVertex{V, E}}())
    root = tree
    pred(root) && push!(result, root)
    for child in root.children
        find(pred, child, result)
    end
    return result
end

function toposort{V, E}(tree::Tree{V, E}, result = Vector{TreeVertex{V, E}}())
    root = tree
    push!(result, root)
    for child in root.children
        toposort(child, result)
    end
    return result
end

function insert!{V, E}(tree::Tree{V, E}, vertexData::V, parentData::V, edgeData::E)
    parentVertex = findfirst(x -> x.vertexData == parentData, tree)
    parentVertex == nothing && error("parent not found")
    vertex = TreeVertex{V, E}(vertexData, parentVertex, edgeData)
    push!(parentVertex.children, vertex)
    return vertex
end

function find_ancestors{V, E}(vertex::TreeVertex{V, E}, result = Vector{TreeVertex{V, E}}())
    push!(result, vertex)
    !isroot(vertex) && find_ancestors(get(vertex.parent), result)
    return result
end

function least_common_ancestor{V, E}(v1::TreeVertex{V, E}, v2::TreeVertex{V, E})
    ancestors1 = find_ancestors(v1);
    ancestors2 = find_ancestors(v2);
    for i = min(length(ancestors1), length(ancestors2)) : -1 : 2
        ancestors1[i] != ancestors2[i] && return ancestors1[i + 1]
    end
    return ancestors1[1]
end

find_ancestors (generic function with 2 methods)

In [14]:
v1 = Tree{Int64, Int32}(0);
v2 = insert!(v1, 1, 0, Int32(1))
toposort(v1)
find(x -> isempty(x.children), v1)
least_common_ancestor(v2, v1)

TreeVertex{Int64,Int32}(1,Nullable(TreeVertex{Int64,Int32}(0,Nullable{TreeVertex{Int64,Int32}}(),Nullable{Int32}(),[TreeVertex{Int64,Int32}(#= circular reference =#)])),Nullable(1),TreeVertex{Int64,Int32}[])

In [ ]:
abstract CacheElement{T}

immutable ImmutableCacheElement{T} <: CacheElement{T}
    t::T
    dependents::Vector{CacheElement{T}}
end

type MutableCacheElement{T} <: CacheElement{T}
    t::T
    dependents::Vector{CacheElement{T}}
    dirty::Bool
    updateFunction::Function
end

get{T}(element::ImmutableCacheElement{T}) = element.t
setdirty{T}(element::ImmutableCacheElement{T}) = 

get{T}(element::MutableCacheElement{T}) = (element.dirty && updateFunction()) || return element.t


type FrameCache
    tree::Tree{CartesianFrame3D, CacheElement{Transform3D}}
    # tree of frames + functions to compute transform to parent, rooted at world
    # transforms on edges
    # computing transform to parent frame can depend on other transforms, which will be computed on the fly;
    # these dependencies will not be stored explicitly
    # memoize all transforms that go up the tree (inverse is very easy to compute)
    # dirty bit for each transform
    # store all fixed transforms and have them never get dirty
end

function setCacheDirty(cache::FrameConfigurationCache)
    # set dirty bit for all non-constant transforms
end

function computeTransform(cache::FrameConfigurationCache, from::CartesianFrame3D, to::CartesianFrame3D)
    # find least common ancestor in dependency graph
    # 
end

In [ ]:
immutable SpatialInertia{T}
    frame::CartesianFrame3D
    moment::Mat{3, 3, T}
    centerOfMass::Vec{3, T}
    mass::T
end

function transform{T}(inertia::SpatialInertia{T}, t::Transform3D{T})
    @assert t.from == inertia.frame
    
    function vector_to_skew_symmetric_squared(a::Vec{3, T})
        aSq = a .* a
        b11 = -aSq[2] - aSq[3]
        b12 = a[1] * a[2]
        b13 = a[1] * a[3]
        b22 = -aSq[1] - aSq[3]
        b23 = a[2] * a[3]
        b33 = -aSq[1] - aSq[2]
        return @fsa([b11 b12 b13; b12 b22 b23; b13 b23 b33])
    end

    J = inertia.moment
    m = inertia.mass
    c = inertia.centerOfMass

    R = Mat(rotationmatrix(t.rot))
    p = t.trans
    
    cnew = R * (c * m)
    Jnew = vector_to_skew_symmetric_squared(cnew)
    cnew += m * p
    Jnew -= vector_to_skew_symmetric_squared(cnew)
    Jnew /= m
    Jnew += R * J * R'
    cnew /= m
    
    return SpatialInertia(t.to, Jnew, cnew, m)
end

In [ ]:
immutable RigidBody
    frame::CartesianFrame3D
    inertia::Nullable{SpatialInertia{Float64}}

    # world body
    RigidBody(name::ASCIIString) = new(CartesianFrame3D(name), null)
    
    # other bodies
    RigidBody(inertia::SpatialInertia{Float64}) = new(inertia.frame, inertia)
end
name(b::RigidBody) = b.frame.name
isroot(b::RigidBody) = isnull(b.inertia)

In [ ]:
immutable SpatialMotionMatrix{N, T}
    body::RigidBody
    base::RigidBody
    frame::CartesianFrame3D
    angular::Mat{3, N, T}
    linear::Mat{3, N, T}
end
typealias SpatialMotionVector{T} SpatialMotionMatrix{1, T}

function transform{N, T}(m::SpatialMotionMatrix{N, T}, t::Transform3D{T})
    @assert m.frame == t.from
    angular = rotate(m.angular, t.rot)
    linear = rotate(m.linear, t.rot)
    for i = 1 : size(m.linear, 2)
        linear(:, i) -= cross(angular(:, i), t.trans)
    end
    return SpatialMotionMatrix(m.body, m.base, t.to, angular, linear)
end

immutable SpatialForceMatrix{N, T}
    body::RigidBody
    base::RigidBody
    frame::CartesianFrame3D
    angular::Mat{3, N, T}
    linear::Mat{3, N, T}
end
typealias SpatialForceVector{T} SpatialForceMatrix{1, T}

function transform{N, T}(f::SpatialForceMatrix{N, T}, t::Transform3D{T})
    @assert f.frame == t.from
    linear = rotate(f.linear, t.rot)
    angular = rotate(f.angular, t.rot)
    for i = 1 : size(f.angular, 2)
        angular(:, i) -= cross(linear(:, i), t.trans)
    end
    return SpatialForceMatrix(f.body, f.base, t.to, angular, linear)
end


In [ ]:
abstract Joint

immutable QuaternionFloatingJoint <: Joint
    name::ASCIIString
end
joint_transform{T}(j::QuaternionFloatingJoint, q::Vector{T}, before::CartesianFrame3D, after::CartesianFrame3D) = Transform3D(Quaternion(q[1], q[2 : 4]))
num_positions(joint::QuaternionFloatingJoint) = 7
num_velocities(joint::QuaternionFloatingJoint) = 6

immutable PrismaticJoint <: Joint
    name::ASCIIString
    translation_axis::Vec{3}
end
joint_transform{T}(j::PrismaticJoint, q::Vector{T}, before::CartesianFrame3D, after::CartesianFrame3D) = Transform3D(after, before, q[1] * j.axis.linear)
num_positions(joint::PrismaticJoint) = 1
num_velocities(joint::PrismaticJoint) = 1

immutable RevoluteJoint <: Joint
    name::ASCIIString
    rotation_axis::Vec{3}
end
joint_transform{T}(j::RevoluteJoint, q::Vector{T}, before::CartesianFrame3D, after::CartesianFrame3D) = Transform3D(after, before, qrotation(j.axis.angular, q[1]))
num_positions(joint::RevoluteJoint) = 1
num_velocities(joint::RevoluteJoint) = 1

In [ ]:
# type Mechanism
#     graph::DiGraph
#     indexToJoint::Dict{Pair{Int64,Int64}, Joint}
#     jointToIndex::Dict{Joint, Pair{Int64,Int64}}
#     joints::Vector{Joint}
#     bodies::Vector{RigidBody}
#     bodyToIndex::Dict{RigidBody, Int64}
    
#     Mechanism() = new(DiGraph(), Dict(), Dict(), Vector(), Vector(), Dict())
# end

# function add_body!(m::Mechanism, b::RigidBody)
#     index = add_vertex!(m.graph)
#     push!(m.bodies, b)
#     m.bodyToIndex[b] = index
#     return b
# end

# function add_joint!(m::Mechanism, j::Joint, predecessor::RigidBody, successor::RigidBody)
#     index = add_edge!(m.graph, m.bodyToIndex[predecessor], m.bodyToIndex[successor])
#     push!(m.joints, j)
#     m.indexToJoint[index] = j
#     m.jointToIndex[j] = index
#     return j
# end

In [ ]:
J = rand(Mat{3, 3, Float64}); J = J * J'
c = rand(Vec{3, Float64})
m = rand()
inertia = SpatialInertia(f3, J, c, m)
m = Mechanism()
world = add_body!(m, RigidBody("world"))
body = add_body!(m, RigidBody("body", inertia))
add_joint!(m, QuaternionFloatingJoint("joint"), world, body)

In [ ]:
transform(inertia, t3)

In [ ]:
type MechanismState{T}
    q::Vector{T}
    v::Vector{T}
end

In [ ]:
bla